## Deliverable 2

### Task:
Reconstruction of both texts using 3 different automatic Python pipeline libraries.

---

### Text 1:
Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safety and greatness in our lives. Hope you too enjoy it as my deepest wishes.  
Thank you for your message to show our words to the doctor, as his next contract checking, to all of us.  
I got this message to see the approved message. In fact, I have received the message from the professor, to show me this, a couple of days ago. I greatly appreciate the full support of the professor for our Springer proceedings publication.

---

### Text 2:
During our final discussion, I told him about the new submission — the one we were waiting for since last autumn, but the updates were confusing as they did not include the full feedback from the reviewer or maybe the editor?  
Anyway, I believe the team, although there was a bit of delay and less communication in recent days, really tried their best for the paper and cooperation. We should be grateful, I mean all of us, for the acceptance and efforts until the Springer link finally came last week, I think.  
Also, kindly remind me, please, if the doctor still plans to edit the acknowledgments section before sending it again. Because I didn’t see that part finalized yet, or maybe I missed it. I apologize if so.  
Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future targets.

---

### Models Used:

1. **First model: Vamsi/T5_Paraphrase_Paws**  
   [Vamsi/T5_Paraphrase_Paws - Hugging Face](https://huggingface.co/Vamsi/T5_Paraphrase_Paws)

2. **Second model: ramsrigouthamg/t5_paraphraser**  
   [ramsrigouthamg/t5_paraphraser - Hugging Face](https://huggingface.co/ramsrigouthamg/t5_paraphraser)

3. **Third model: prithivida/grammar_error_correcter_v1**  
   [prithivida/grammar_error_correcter_v1 - Hugging Face](https://huggingface.co/prithivida/grammar_error_correcter_v1)  
   This model was used for correcting grammatical and spelling errors.

4. **Fourth model: eugenesiow/bart-paraphrase**  
   [eugenesiow/bart-paraphrase - Hugging Face](https://huggingface.co/eugenesiow/bart-paraphrase)  
   Paraphrasing/reconstruction using the BART architecture.

5. **Fifth model: stanford-oval/paraphraser-bart-large**  
   [stanford-oval/paraphraser-bart-large - Hugging Face](https://huggingface.co/stanford-oval/paraphraser-bart-large)  
   Sentence-level paraphrasing using BART, based on back-translation.

6. **Sixth model: liamcripwell/ctrl44-simp**  
   [liamcripwell/ctrl44-simp - Hugging Face](https://huggingface.co/liamcripwell/ctrl44-simp)


In [ ]:
from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
)
import torch

# Models to run
models = [
    #"Vamsi/T5_Paraphrase_Paws",
    #"ramsrigouthamg/t5_paraphraser",
    #"prithivida/grammar_error_correcter_v1",
    #"eugenesiow/bart-paraphrase",
    #"stanford-oval/paraphraser-bart-large",
    "liamcripwell/ctrl44-simp",
    
]

texts = {
    "text1": (
        "Today is our dragon boat festival, in our Chinese culture, "
        "to celebrate it with all safe and great in our lives. "
        "Hope you too, to enjoy it as my deepest wishes. "
        "Thank your message to show our words to the doctor, as his next contract checking, to all of us. "
        "I got this message to see the approved message. In fact, I have received the message from the "
        "professor, to show me this a couple of days ago. I am very appreciated the full support of the "
        "professor, for our Springer proceedings publication."
    ),
    "text2": (
        "During our final discuss, I told him about the new submission — the one we were waiting since "
        "last autumn, but the updates was confusing as it not included the full feedback from reviewer or "
        "maybe editor? Anyway, I believe the team, although bit delay and less communication at recent days, they really "
        "tried best for paper and cooperation. We should be grateful, I mean all of us, for the acceptance "
        "and efforts until the Springer link came finally last week, I think. "
        "Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before "
        "he sending again. Because I didn’t see that part final yet, or maybe I missed, I apologize if so. "
        "Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future targets."
    )
}

device = 0 if torch.cuda.is_available() else -1

for text_name, text in texts.items():
    print(f"\n\n######## {text_name.upper()} ########\n")
    for model_name in models:
        print(f"-- Model: {model_name} --")

        # 1) Controlled simplification with ctrl44-simp
        if model_name == "liamcripwell/ctrl44-simp":
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
            model     = AutoModelForSeq2SeqLM.from_pretrained(model_name)
            paraphraser = pipeline(
                "text2text-generation",
                model=model,
                tokenizer=tokenizer,
                device=device
            )
            # prepend <para> to request paraphrase/simplification
            controlled_input = "<para> " + text
            outputs = paraphraser(
                controlled_input,
                max_length=512,
                num_return_sequences=3,
                do_sample=True,
                top_k=50,
                top_p=0.95
            )
            variants = [out['generated_text'] for out in outputs]

        # 2) GPT-Neo causal LM paraphrasing
        elif "gpt-neo" in model_name:
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model     = AutoModelForCausalLM.from_pretrained(model_name)
            paraphraser = pipeline(
                "text-generation",
                model=model,
                tokenizer=tokenizer,
                device=device
            )
            prompt = f"Rephrase clearly: \"{text}\""
            outputs = paraphraser(
                prompt,
                max_length=len(tokenizer(prompt)["input_ids"]) + 200,
                num_return_sequences=3,
                do_sample=True,
                top_k=50,
                top_p=0.95
            )
            variants = [out['generated_text'][len(prompt):].strip() for out in outputs]

        # 3) Sequence classification (e.g., ambiguity detection)
        elif "srs_ambiguity_model" in model_name:
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model     = AutoModelForSequenceClassification.from_pretrained(model_name)
            classifier = pipeline(
                "text-classification",
                model=model,
                tokenizer=tokenizer,
                device=device
            )
            preds = classifier(text)
            print("Ambiguity classification:", preds, "\n")
            continue

        # 4) Generic Seq2Seq paraphrase / simplification
        else:
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
            model     = AutoModelForSeq2SeqLM.from_pretrained(model_name)
            paraphraser = pipeline(
                "text2text-generation",
                model=model,
                tokenizer=tokenizer,
                device=device
            )
            outputs = paraphraser(
                text,
                max_length=512,
                num_return_sequences=3,
                do_sample=True,
                top_k=50,
                top_p=0.95
            )
            variants = [out['generated_text'] for out in outputs]

        # Print out generated variants
        for i, variant in enumerate(variants, start=1):
            print(f"Variant {i}:\n{variant}\n")




######## TEXT1 ########

-- Model: liamcripwell/ctrl44-simp --


Device set to use cpu


Variant 1:
  Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too, to enjoy it as my deepest wishes. Thank your message to show our words to the doctor, as his next contract checking, to all of us. I got this message to see the approved message. In fact, I have received the message from the professor, to show me this a couple of days ago. I am very appreciated the full support of the professor for our Springer proceedings publication.

Variant 2:
   Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too, to enjoy it as my deepest wishes. Thank your message to show our words to the doctor, as his next contract checking, to all of us. I got this message to see the approved message. In fact, I have received the message from the professor, to show me this a couple of days ago. I am very appreciated the full support of the professor.

Variant 3:
   

Device set to use cpu


Variant 1:
 During our final discuss, I told him about the new submission — the one we were waiting since last autumn, but the updates was confusing as it not included the full feedback from reviewer or maybe editor? Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation. We should be grateful, I mean all of us, for the acceptance and efforts until the Springer link came finally last week, I think. Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before he sending again. Because I didn’t see that

Variant 2:
   During our final discuss, I told him about the new submission — the one we were waiting since last autumn, but the updates was confusing as it not included the full feedback from reviewer or maybe editor? Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation. We should be grate

## Results

---

### Text 1:

Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safety and greatness in our lives. Hope you too enjoy it as my deepest wishes.  
Thank you for your message to show our words to the doctor, as his next contract checking, to all of us.  
I got this message to see the approved message. In fact, I have received the message from the professor, to show me this, a couple of days ago. I greatly appreciate the full support of the professor for our Springer proceedings publication.

---

#### Model: Vamsi/T5_Paraphrase_Paws

- **Variant 1:**  
  Today is our Dragon Boat Festival in our Chinese culture to celebrate it with all safe and great in our lives and hope you as our deepest wishes to enjoy it as well. I received this message from the professor to show me this a couple of days ago. I am very grateful for the support from the professor for the publication of our Springer proceedings.

- **Variant 2:**  
  Today is our Dragon Boat Festival in our Chinese culture to celebrate it with all safe and great in our lives. Hope you too enjoy it as my deepest wishes. I received this message to see the approved message from the professor, a couple of days ago, and am very grateful for the full support from the professor for our Springer proceedings publication.

- **Variant 3:**  
  Today is our Dragon Boat Festival in our Chinese culture to celebrate with all safe and great in our lives. Hope you enjoy it as my deepest wishes. Thank you for your message to show the doctor our words to help him with his next contract check.

---

#### Model: ramsrigouthamg/t5_paraphraser

- **Variant 1:**  
  Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too enjoy it as my deepest wishes. Thank you for seeing the approved message. In fact, I received the message from the professor to show me this a couple of days ago.

- **Variant 2:**  
  Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too enjoy it as my deepest wishes. I got this message to see the approved message. In fact, I have received the message from the professor to show me this a couple of days ago. I am very appreciative of the full support of the professor for our Springer proceedings publication.

- **Variant 3:**  
  I got this message to see our words to the doctor, as his next contract checking, to all of us. Hope you too enjoy it with all safe and great in our lives. Thank you for your message to show our words to the doctor as his next contract check. I received this message to see the approved message. In fact, I received the message from the professor to show me this a couple of days ago.

---

#### Model: prithivida/grammar_error_correcter_v1

- **Variant 1:**  
  Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too enjoy it as my deepest wishes. Thank you for your message to the doctor, with his next contract checking, to all of us. I got this message to see the approved...

- **Variant 2:**  
  Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safety and great in our lives. Hope you too celebrate it with all that is safe and great in our lives. I got the message to show our words to the doctor, as his next contract checking, to all of...

- **Variant 3:**  
  Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safety and great in our lives. Thanks for your message to show our words to the doctor, and on his next contract checking, to all of us. I have received the message to see the approved message. I am very appreciative of the...

---

#### Model: eugenesiow/bart-paraphrase

- **Variant 1:**  
  Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Thank you for your message to show our words to the doctor, as his next contract check, to all of us. I am very appreciative of the full support of the professor for our Springer proceedings publication.

- **Variant 2:**  
  Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Thank you for your message to show our words to the doctor as his next contract check, to all of us. I am very appreciative of the full support of the professor for our Springer proceedings publication.

- **Variant 3:**  
  Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Thank you for your message to show our words to the doctor, as his next contract check, to all of us. I am very appreciative of the full support of the professor for our Springer proceedings publication.

---

#### Model: stanford-oval/paraphraser-bart-large

- **Variant 1:**  
  And I actually received that report a few days ago from the professor, with the full support of the faculty, to celebrate safely and happily at the Springer Convention, as I wish we all did. I hope you will take a moment to appreciate this report, from the doctor, to see what's happening.

- **Variant 2:**  
  Actually, I got a message a few days ago from the professor, to celebrate with all the safe and lovely people in our lives. I hope you will all also enjoy it. Thank you for your message for showing our words to the doctor as your next screening of our contract.

- **Variant 3:**  
  Thank you for your message — the professor is coming to see me for the festival today, our Dragon Boat Festival in Chinese culture, to celebrate with everyone safe and well in our lives, and I hope you all enjoy it as much as I do.

---

#### Model: liamcripwell/ctrl44-simp

- **Variant 1:**  
  Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too enjoy it as my deepest wishes. Thank you for your message to show our words to the doctor, as his next contract check, to all of us. I got this message to see the approved message. In fact, I have received the message from the professor to show me this a couple of days ago. I am very appreciative of the full support of the professor for our Springer proceedings publication.

- **Variant 2:**  
  Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too enjoy it as my deepest wishes. Thank you for your message to show our words to the doctor, as his next contract check, to all of us. I got this message to see the approved message. In fact, I have received the message from the professor to show me this a couple of days ago. I am very appreciative of the full support of the professor for our Springer proceedings publication.

- **Variant 3:**  
  Today is our Dragon Boat Festival, in our Chinese culture, to celebrate it with all safe and great in our lives. Hope you too enjoy it as my deepest wishes. Thank you for your message to show our words to the doctor, as his next contract check, to all of us. I got this message to see the approved message. In fact, I have received the message from the professor to show me this a couple of days ago. I am very appreciative of the full support of the professor.

---

### Text 2:

During our final discussion, I told him about the new submission — the one we were waiting for since last autumn, but the updates were confusing as they did not include the full feedback from the reviewer or maybe the editor?  
Anyway, I believe the team, although there was a bit of delay and less communication in recent days, really tried their best for the paper and cooperation. We should be grateful, I mean all of us, for the acceptance and efforts until the Springer link finally came last week, I think.  
Also, kindly remind me, please, if the doctor still plans to edit the acknowledgments section before sending it again. Because I didn’t see that part finalized yet, or maybe I missed it. I apologize if so.  
Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future targets.

---

#### Model: Vamsi/T5_Paraphrase_Paws

- **Variant 1:**  
  I told him about the new submission during the final discussion — the one we were waiting for since last fall, but the updates were confusing as it was not the full feedback from the reviewer or maybe editor? Anyway, I think the team really tried their best for the paper and cooperation. We should be grateful — I mean all of us — if the doctor still plans to edit the acknowledgments section before he sends it again, because I didn’t see that part finalized yet or maybe I missed it. I apologize if so.

- **Variant 2:**  
  I told him about the new submission during our final discussion, which we had been waiting for since last autumn, but the updates were confusing because they did not include the full feedback from the reviewer or maybe the editor? Anyway, we should all be grateful for the acceptance and efforts until the Springer link finally came last week, I think. Also, kindly remind me — I didn’t see the final part yet or maybe I missed it. Let’s all celebrate the outcome with strong coffee and future goals.

- **Variant 3:**  
  At the end of our discussion, I told him about the new submission — one we had been waiting for since autumn of last year, but the updates were confusing as they did not include the full feedback from the reviewer or maybe the editor? Anyway, I believe the team did their best for the paper and cooperation. We should all be grateful, if the doctor still plans to edit the acknowledgments section before sending again — maybe I just didn’t see that part yet.

---

#### Model: ramsrigouthamg/t5_paraphraser

- **Variant 1:**  
  I told him about the new submission — the one we were waiting for since last autumn, but the updates were confusing as they didn’t include the full feedback from the reviewer or maybe the editor? Anyway, I believe the team, despite some delay and less communication recently, really tried their best for the paper and cooperation. We should all be grateful for the acceptance and efforts until the Springer link finally came last week, I think.

- **Variant 2:**  
  During our final discussion, I told him about the new submission — the one we were waiting for since last autumn, but the updates were confusing as they didn’t include the full feedback from the reviewer or maybe the editor? Anyway, I believe the team, despite a bit of delay and less communication recently, really tried their best for the paper and cooperation. We should all be grateful for the acceptance and efforts until the Springer link finally came last week, I think. Also, kindly remind me, please, if the doctor still plans to...

- **Variant 3:**  
  I want to thank the Springer team for their cooperation, even when I wasn't yet aware of the final deadline. Let’s make sure everyone is safe.

---

#### Model: prithivida/grammar_error_correcter_v1

- **Variant 1:**  
  Finally, I told Mr. Kuhlman about the new submission — the one we were waiting for since last autumn, but the updates were confusing as they did not include the full feedback from the reviewers, or maybe the editor? Anyway, I think the team, although...

- **Variant 2:**  
  Overall, let us make sure everyone is safe and celebrate the outcome with strong coffee and future targets. We’ll keep you posted with the latest reprint — the one we were waiting for since last autumn, but the updates were confusing, as they did not include the full feedback from the reviewers...

- **Variant 3:**  
  After our final discussion, I told him about the new submission — the one we were waiting for since last autumn, but the updates were confusing as they did not include the full feedback from the reviewer or maybe the editor? Anyway, I believe the team, although a bit delayed and with less communication of late...

---

#### Model: eugenesiow/bart-paraphrase

- **Variant 1:**  
  During our final discussion, I told him about the new submission — the one we were waiting for since last autumn, but the updates were confusing as they did not include the full feedback from the reviewer or maybe the editor. We should all be grateful, especially if the doctor still plans to edit the acknowledgments section before sending it again. I didn’t see that part finalized yet, or maybe I missed it. I apologize if so.

- **Variant 2:**  
  During our final discussion, I told him about the new submission — the one we were waiting for since last autumn, but the updates were confusing as they did not include the full feedback from the reviewer or maybe the editor. We should all be grateful for the acceptance and efforts until the Springer link finally came last week. I apologize if so. Also, kindly remind me if the doctor still plans to edit the acknowledgments section before sending it again. I believe the team, despite a bit of delay and less communication recently...

- **Variant 3:**  
  During our final discussion, I told him about the new submission — the one we were waiting for since last autumn, but the updates were confusing as they did not include the full feedback from the reviewer or maybe the editor. We should all be grateful for the acceptance and efforts until the Springer link finally came last week. I apologize if so. Also, kindly remind me if the doctor still plans to edit the acknowledgments section before sending it again. I believe the team, despite some delay and less communication recently...

---

#### Model: stanford-oval/paraphraser-bart-large

- **Variant 1:**  
  When I was finalizing the talk last week, I felt we should all be grateful. We know she submitted something last year, but the update was confusing because it didn’t yet include the full feedback from the reviewer or maybe the editor. It might have gotten muddled in the last few days, so please forgive me if that’s the case.

- **Variant 2:**  
  As we ended our last talk, I think we should also thank the team for the acceptance and for making one last push last week. The feedback was confusing, as it missed a final part from the reviewer or maybe from someone’s notes. I apologize if we overreacted. Let’s celebrate with our meal and drinks — and look ahead to new goals.

- **Variant 3:**  
  And we should all, I think, be grateful to the team for accepting and eventually submitting that new submission last week — the one we’ve been waiting for since last fall — but the update was puzzling because it didn’t include all the feedback from the reviewers or the potential editor. I apologize if it was just a timing issue.

---

#### Model: liamcripwell/ctrl44-simp

- **Variant 1:**  
  During our final discussion, I told him about the new submission — the one we were waiting for since last autumn, but the updates were confusing as they didn’t include the full feedback from the reviewer or maybe the editor. Anyway, I believe the team, though slightly delayed and with less communication recently, really tried their best for the paper and cooperation. We should all be grateful for the acceptance and efforts until the Springer link finally came last week, I think. Also, please remind me if the doctor still plans to edit the acknowledgments section before sending again. Because I didn’t see that part...

- **Variant 2:**  
  During our final discussion, I told him about the new submission — the one we were waiting for since last autumn, but the updates were confusing as they didn’t include the full feedback from the reviewer or maybe the editor. Anyway, I believe the team, despite some delay and less communication lately, really tried their best for the paper and cooperation. We should all be grateful for the acceptance and efforts until the Springer link finally came last week, I think. Also, kindly remind me if the doctor still plans to edit the acknowledgments section before sending again. Because I didn’t see that part...

- **Variant 3:**  
  During our final discussion, I told him about the new submission — the one we were waiting for since last autumn, but the updates were confusing as they didn’t include the full feedback from the reviewer or maybe the editor. Anyway, I believe the team, though slightly delayed and with less communication recently, really tried their best for the paper and cooperation. We should all be grateful for the acceptance and efforts until the Springer link finally came last week, I think. Also, kindly remind me if the doctor still plans to edit the acknowledgments section before sending again. Because I didn’t see that part...

---
